In [1]:
import numpy as np
import pandas as pd
from scipy.stats import qmc # Latin Hypercube !
from scipy.interpolate import interp1d 
from tqdm import tqdm
import time

import sys, os, pathlib

NOTEBOOK_DIR = pathlib.Path(os.getcwd())  
ROOT = NOTEBOOK_DIR.parents[0]  
sys.path.insert(0, str(ROOT))
from src.Heston_FFT import heston_price_fft


n_samples = 25_000

# Market-level constants
S0 = 105.0
T  = 1.0
r  = 0.05
q  = 0.0

alpha = 1.5
n = 4096
eta = 0.1
lam = 2.0*np.pi / (n * eta)  


def sample_heston_lhs(
    n_samples: int,
    seed: int = 42,
    moneyness_bounds=(0.7, 1.3)
) -> pd.DataFrame:
    """
    Generate n_samples plausible Heston parameters via Latin Hypercube Sampling (LHS),
    and also sample option strikes through moneyness m = K / S0.

    Constraints:
      - v0, theta in [0.01, 0.09]   ~ variance 1%–9% (vol ~10%–30%)
      - kappa in [0.5, 5.0]         ~ reasonable mean reversion
      - sigma_v in [0.10, 1.00]     ~ realistic vol-of-vol
      - rho in [-0.95, -0.10]       ~ typically negative corr
      - Feller: 2*kappa*theta >= sigma_v^2
      - moneyness m in [m_low, m_high] with K = S0 * m

    Parameters
    ----------
    n_samples : int
        Number of samples to return after filtering by constraints.
    seed : int
        Random seed for reproducibility.
    moneyness_bounds : tuple(float, float)
        Lower and upper bounds for m = K/S0.

    Returns
    -------
    pd.DataFrame
        Columns: ['S0','K','T','r','q','v0','kappa','theta','sigma_v','rho'].
    """
    # BOUNDS (l_bounds, u_bounds) order: v0, kappa, theta, sigma_v, rho, moneyness
    m_low, m_high = moneyness_bounds
    if not (m_low > 0 and m_high > m_low):
        raise ValueError("moneyness_bounds must satisfy 0 < m_low < m_high.")

    l_bounds = np.array([0.01, 0.5, 0.01, 0.10, -0.95, m_low], dtype=float)
    u_bounds = np.array([0.09, 5.0, 0.09, 1.00, -0.10, m_high], dtype=float)

    collected = []
    attempt = 0

    # Repeat in batches until we have enough valid samples
    while len(collected) < n_samples and attempt < 20:
        # Oversample to compensate for filtering
        batch_size = int(np.ceil((n_samples - len(collected)) * 2.0))
        sampler = qmc.LatinHypercube(d=6, seed=seed + attempt)
        U = sampler.random(batch_size)
        X = qmc.scale(U, l_bounds, u_bounds)  # shape [batch_size, 6]

        v0, kappa, theta, sigma_v, rho, m = (
            X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5]
        )

        # Feller condition to ensure variance stays strictly positive
        feller_ok = (2.0 * kappa * theta) >= (sigma_v ** 2)

        good = feller_ok
        X_ok = X[good]

        # Append valid rows until reaching n_samples
        for row in X_ok:
            collected.append(row)
            if len(collected) >= n_samples:
                break

        attempt += 1

    if len(collected) < n_samples:
        raise RuntimeError(
            "Could not generate enough samples satisfying constraints. "
            "Relax bounds slightly or increase attempts."
        )

    params = np.vstack(collected)[:n_samples]
    df = pd.DataFrame(params, columns=["v0", "kappa", "theta", "sigma_v", "rho", "m"])

    # Compute K from moneyness and add market columns
    df["S0"] = S0
    df["K"]  = (df["m"] * S0).astype(float)
    df["T"]  = T
    df["r"]  = r
    df["q"]  = q

    # Reorder columns
    df = df[["S0","K","T","r","q","v0","kappa","theta","sigma_v","rho"]]

    return df

df_first = sample_heston_lhs(n_samples, seed=123)

In [2]:
def price_heston_fft_df(
    df: pd.DataFrame,
    option_type: str = "call",
    alpha: float = 1.5,
    N: int = 4096,
    eta: float = 0.05,
) -> pd.DataFrame:
    """
    Aggiunge la colonna 'price' con il prezzo Heston per ciascuna riga di df,
    utilizzando la funzione heston_price_fft già importata.

    Richiede che df contenga le colonne:
    ['S0','K','T','r','q','v0','kappa','theta','sigma_v','rho'].
    """
    required = ["S0","K","T","r","q","v0","kappa","theta","sigma_v","rho"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Mancano colonne richieste nel DataFrame: {missing}")

    prices = []
    for _, row in df.iterrows():
        try:
            p = heston_price_fft(
                S=float(row["S0"]),
                K=float(row["K"]),
                T=float(row["T"]),
                r=float(row["r"]),
                v0=float(row["v0"]),
                kappa=float(row["kappa"]),
                theta=float(row["theta"]),
                sigma_v=float(row["sigma_v"]),
                rho=float(row["rho"]),
                option_type=option_type,
                q=float(row["q"]),
                alpha=alpha,
                N=N,
                eta=eta,
            )
        except Exception:
            # Se qualcosa va storto per una riga (es. instabilità numerica), metti NaN
            p = np.nan
        prices.append(p)

    out = df.copy()
    out["price"] = prices
    return out


df = price_heston_fft_df(df_first)
df.head()

,S0,K,T,r,q,v0,kappa,theta,sigma_v,rho,price
0,105.0,110.898297,1.0,0.05,0.0,0.083975,2.489625,0.073320,0.463597,-0.850519,10.714906
1,105.0,114.461866,1.0,0.05,0.0,0.056668,2.359803,0.044652,0.189295,-0.465270,7.439766
2,105.0,101.842912,1.0,0.05,0.0,0.062475,3.146269,0.078854,0.522231,-0.934648,15.447042
3,105.0,81.240139,1.0,0.05,0.0,0.034656,3.114431,0.064181,0.357214,-0.212404,28.830986
4,105.0,97.271398,1.0,0.05,0.0,0.045612,0.607602,0.070018,0.289997,-0.603681,16.758359


In [3]:
# check
price_fft_example = heston_price_fft(S=105, K=97.271398, T=1.0, r=0.05, v0=0.045619, kappa=0.6076015, theta=0.070017, sigma_v=0.289997, rho=-0.60368,
                             option_type="call", q=0.0, alpha=alpha, N=n, eta=eta)

price_fft_example

16.75272652332878

In [ ]:
import os
os.makedirs("data", exist_ok=True)
#df.to_csv("data/heston_fft_prices.csv", index=False)